## 請手刻gradient descent實作logistic regression

In [37]:
import numpy as np

##### read X_train and Y_train

In [38]:
import csv

# 106 columns

X_train = []
Y_train = []

with open('data/X_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        X_train.append([float(i) for i in row] + [1.0])
        
        '''continuous_idxs = []
        for idx2, num in enumerate(row):
            if float(num) > 1:
                continuous_idxs.append(idx2)
        print(continuous_idxs)'''

X_train = np.array(X_train)
for col in [0,1,3,4,5]: # [0,1,3,4,5] only, shouldn't normalize one-hot columns, will be affected by data
    if np.std(X_train[:,col]) != 0:
        X_train[:,col] = np.divide((X_train[:,col] - np.average(X_train[:,col])), np.std(X_train[:,col]))
# [[data1]
#  [data2]]

with open('data/Y_train.csv', 'rt', encoding='big5') as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # skip headings
    for idx, row in enumerate(reader):
        Y_train.append(float(row[0]))

Y_train = np.array(Y_train)
# [y1 y2]


In [39]:
print(len(X_train[0]))
# print(X_train[0:2])
# print(X_train[0:3])
print(X_train[7])
print(Y_train[7])

107
[ 0.98373415  0.18819463  1.         -0.14592048 -0.21665953  0.36951938
  0.          0.          0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          1.
  0.          0.          0.          0.          0.          0.          1.
  0.          0.          0.          0.          0.          0.          0.
  1.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          1.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          1.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.

##### define sigmoid function for numpy

In [40]:
def sigmoid(x): # this sigmoid works with numpy arrays
    res =  1.0 / (1 + np.exp(-x))
    return np.clip(res, 0.00000000000001, 0.99999999999999)

##### start training, split X_train into X_real_train and X_valid

##### currently not caring about validaton set, directly train with X_real_train = X_train

###### accuracy stuck at 0.82

In [41]:
# optimizations to consider when hand-crafting everything: feature scaling (done), l1 l2 norm

# grab validation set
import random
val_idxs = random.sample(range(0, len(X_train)), int(len(X_train) * 0.2))
X_valid = X_train[val_idxs]
Y_valid = Y_train[val_idxs]

'''X_real_train = np.delete(X_train, val_idxs, 0)
Y_real_train = np.delete(Y_train, val_idxs, 0)'''
X_real_train = X_train
Y_real_train = Y_train

# old batch = 50 #len(X_real_train)
batch = len(X_real_train)
lr = 0.002
# decay = 1
my_lambda = 0 # l2 norm

learning_rates = [1.0] # for adagrad


# initialize weights
weights = np.zeros(len(X_train[0])) # we already appended bias to X_train

#===================saving previous weights======================
# weights = weights_saved

for epoch in range(0,5000):
    # randomly decide training order in this epoch
    # [41,6,0,231...]
    random_idxs = random.sample(range(0, len(X_real_train)), len(X_real_train))
    for i in range(0, len(random_idxs), batch):
        X_batch = X_real_train[random_idxs[i: min(i+batch, len(random_idxs))]]
        Y_batch = Y_real_train[random_idxs[i: min(i+batch, len(random_idxs))]]
        Y_pred = np.dot(X_batch, weights)
        Y_pred = sigmoid(Y_pred)
        Y_pred = np.around(Y_pred)

        # get gradients and update weights
        gradient = np.zeros(len(X_train[0]))
        for idx_batch in range(0, len(X_batch)):
            l2 = 2 * my_lambda * np.append(weights[:-1],0)
            gradient += - ((Y_batch[idx_batch] - Y_pred[idx_batch]) * X_batch[idx_batch] + l2) / batch

        adagrad_denom = np.sqrt(sum(np.array(learning_rates) ** 2))
        new_lr = lr / adagrad_denom
        learning_rates += new_lr
        # print(new_lr)
        weights = weights - new_lr * gradient # * pow(decay, epoch)
            
    if epoch % 10 == 9:
        train_pred = np.dot(X_real_train, weights)
        train_pred = sigmoid(train_pred)
        train_pred = np.around(train_pred)
        print(train_pred[0:20])
        print(Y_real_train[0:20])
        print(epoch+1, 'training accuracy', np.sum(train_pred == Y_real_train) / (len(Y_real_train)))

        valid_pred = np.dot(X_valid, weights)
        valid_pred = sigmoid(valid_pred)
        valid_pred = np.around(valid_pred)
        print(epoch+1, 'validation accuracy', np.sum(valid_pred == Y_valid) / (len(Y_valid)))

[ 0.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  1.  1.  0.  0.
  1.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.
  0.  1.]
10 training accuracy 0.733085593194
10 validation accuracy 0.726504914005
[ 0.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.
  1.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.
  0.  1.]
20 training accuracy 0.763612911151
20 validation accuracy 0.757371007371
[ 0.  1.  0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.
  1.  1.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.
  0.  1.]
30 training accuracy 0.801940972329
30 validation accuracy 0.793611793612
[ 0.  1.  0.  0.  1.  1.  0.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.
  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  0.  1.  0.  0.  0.
  0.  1.]
40 training accuracy 0.827370166764
40 validation accuracy 0.819563882064
[ 0.  1.  0.  0.  1.  1.  0.  1.  1.  1.

KeyboardInterrupt: 

In [36]:
# 0.82
'''weights_saved = weights
print(weights_saved)
np.save('weights_logistic', weights_saved)'''

"weights_saved = weights\nprint(weights_saved)\nnp.save('weights_logistic', weights_saved)"

In [33]:
X_test = []

with open('data/X_test.csv', 'rt', encoding='big5') as infile:
    reader = csv.reader(infile)

    row1 = next(reader) # skip headings
    for row in reader:
        X_test.append([float(i) for i in row] + [1.0])
            
X_test = np.array(X_test)
for col in [0,1,3,4,5]:
    if np.std(X_test[:,col]) != 0:
        X_test[:,col] = np.divide((X_test[:,col] - np.average(X_test[:,col])), np.std(X_test[:,col]))

pred = np.dot(X_test, weights)
pred = sigmoid(pred)
pred = np.around(pred)
print(pred[0:100])

with open('data/submission.csv', 'wt') as outfile:
    test_writer = csv.writer(outfile)
    test_writer.writerow(['id','label'])
    
    counter = 0
    for num in pred:
        counter += 1
        test_writer.writerow([str(counter),int(num)])
    
            

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.
  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
